### **Extracción Data - Autos**

In [ ]:
## Librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from tqdm import tqdm  # pip install tqdm
import datetime as dt
import re

In [84]:
# enlace = "https://neoauto.com/venta-de-autos-seminuevos"
enlace = "https://neoauto.com/venta-de-autos-seminuevos?page=43"
# enlace = "https://neoauto.com/venta-de-autos-nuevos?page=1"

## Driver Setup

In [85]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()
driver.get(enlace)

In [77]:
resultados = driver.find_element(By.CLASS_NAME, "s-results__count")
print(resultados.text)

860 Resultados


extraccion de datos

In [78]:
cards = 'Hyundai Accent 2024\nAUTO NUEVO'
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2024\nAUTO NUEVO',
        #  'Hyundai Creta Grand 2025\nAUTO NUEVO',
        #  'Ford F-150 2024\nAUTO NUEVO',
        #  'Hyundai Grand I10 2024\nAUTO NUEVO',
        #  'Hyundai Tucson 2025\nAUTO NUEVO'
         

tarejtas = cards.split("\n", 1)[0]
tarejtas

'Hyundai Accent 2024'

In [79]:

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]

encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

tiutlo_auto = []
marca_auto = []
modelo_auto = []
año_auto = []

for enc in encabezado:
    titulo = enc.text
    titulo_ = titulo.split("\n", 1)[0].strip()
    for brand in BRANDS:
        if brand in titulo_:
            
            año = titulo_.split()[-1]
            modelo = titulo_.replace(brand, "").replace(año, "").strip()

            tiutlo_auto.append(titulo_)
            marca_auto.append(brand)    
            modelo_auto.append(modelo)
            año_auto.append(año)
            
modelo_auto

['Sorento',
 'Discovery',
 'Compass',
 'Creta Grand',
 'page',
 'Sportage',
 'Cs55',
 'Jetta',
 'Amarok',
 'Forester',
 'Corolla Cross',
 'Urvan',
 'Yaris Cross',
 'Creta Grand']

In [80]:
import re

BRANDS = [
    "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
    "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
    "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
    "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
    "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
]
brand_re = re.compile(r'^(?P<brand>' + '|'.join(map(re.escape, sorted(BRANDS, key=len, reverse=True))) + r')\b', re.I)
year_re  = re.compile(r'((?:19|20)\d{2})\s*$')
KEEP_UPPER = {"BMW","BYD","GMC","MG","RAM","DFSK"}

def parse_card_title(raw: str):
    # 1) Quédate con la primera línea (títulos vienen como "… 2024\nAUTO NUEVO")
    title = raw.split("\n", 1)[0].strip()

    # 2) Año al final
    m_year = year_re.search(title)
    anio = int(m_year.group(1)) if m_year else None
    name = year_re.sub("", title).strip()

    # 3) Marca al inicio (incluye marcas de 2 palabras)
    m_brand = brand_re.match(name)
    if m_brand:
        brand_raw = m_brand.group("brand")
        marca = brand_raw if brand_raw.upper() in KEEP_UPPER else brand_raw.title()
        modelo = name[m_brand.end():].strip()
    else:
        parts = name.split(maxsplit=1)
        marca = (parts[0].upper() if parts and parts[0].upper() in KEEP_UPPER
                 else (parts[0].title() if parts else ""))
        modelo = parts[1] if len(parts) > 1 else ""

    return {"titulo": title, "marca": marca, "modelo": modelo, "anio": anio}

cards = ['Hyundai Accent 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Tucson 2024\nAUTO NUEVO',
         'Hyundai Creta Grand 2025\nAUTO NUEVO',
         'Ford F-150 2024\nAUTO NUEVO',
         'Hyundai Grand I10 2024\nAUTO NUEVO',
         'Hyundai Tucson 2025\nAUTO NUEVO']

parsed = [parse_card_title(s) for s in cards]
for r in parsed:
    print(r)

{'titulo': 'Hyundai Accent 2024', 'marca': 'Hyundai', 'modelo': 'Accent', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2024', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2024}
{'titulo': 'Hyundai Creta Grand 2025', 'marca': 'Hyundai', 'modelo': 'Creta Grand', 'anio': 2025}
{'titulo': 'Ford F-150 2024', 'marca': 'Ford', 'modelo': 'F-150', 'anio': 2024}
{'titulo': 'Hyundai Grand I10 2024', 'marca': 'Hyundai', 'modelo': 'Grand I10', 'anio': 2024}
{'titulo': 'Hyundai Tucson 2025', 'marca': 'Hyundai', 'modelo': 'Tucson', 'anio': 2025}


In [81]:
precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
precio_auto = []
for prec in precio:
    costo = prec.text
    precio_auto.append(costo)
    
precio_auto[0]

'US$ 27,900'

In [82]:
tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
tag_ = []
for det in tag:
    descripcion = det.text
    tag_.append(descripcion)

tag_[0]

'UNICO DUEÑO'

In [86]:

detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

detalle_auto = []
combustible_auto = []
transmision_auto = []
caja_auto = []
kilometraje_auto = []
ubicacion_auto = []
descripcion_short_auto = []

detalles_autos = []

for detalle in detalles:
    
    detalle_text = detalle.text
    # print(info)
    
    # normaliza y separa líneas no vacías
    t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
    lines = [l.strip() for l in t.splitlines() if l.strip()]

    # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
    combustible = transmision_raw = None
    if lines:
        p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
        combustible = p0[0].title() if p0 else None
        transmision_raw = p0[1] if len(p0) > 1 else None

    # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
    tipo_transmision = caja = None
    if transmision_raw:
        parts = [p.strip() for p in transmision_raw.split("-")]
        tipo_transmision = parts[0]
        caja = parts[1] if len(parts) > 1 else None

    # 2) kilometraje (busca en todo el texto)
    km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
    kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

    # 3) ubicación (primera línea con coma que no sea "Kms")
    ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

    # 4) “modelo/etiqueta” (última línea)
    modelo_line = lines[-1] if lines else None
    
    # salidas
    detalle_auto.append(detalle_text)
    combustible_auto.append(combustible)
    transmision_auto.append(tipo_transmision)
    caja_auto.append(caja)
    kilometraje_auto.append(kilometraje_km)
    ubicacion_auto.append(ubicacion)
    
    detalles_autos.append({
        "detalle": detalle_text,
        "combustible": combustible,
        "transmision": tipo_transmision,
        "caja": caja,
        "kilometraje_km": kilometraje_km,
        "ubicacion": ubicacion,
        "descripcion_short": modelo_line
    })
#     descripcion_short_auto.append(modelo_line)

detalles_autos

[{'detalle': 'Gasolina|Automática\nKilometraje:\n40,859 Kms\nLima, Lima',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 40859,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Lima, Lima'},
 {'detalle': 'Gasolina|Automática\nKilometraje:\n70,126 Kms\nLima, Lima',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 70126,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Lima, Lima'},
 {'detalle': 'Gasolina|Automática\nKilometraje:\n62,285 Kms\nLima, Lima',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 62285,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Lima, Lima'},
 {'detalle': 'Gasolina|Automática\nKilometraje:\n19,875 Kms\nLima, Lima',
  'combustible': 'Gasolina',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje_km': 19875,
  'ubicacion': 'Lima, Lima',
  'descripcion_short': 'Lima, Lima'},
 {'detalle': 'Gasolina|M

In [168]:
enlaces = []
enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
for link in enlace:
    url = link.get_attribute("href")    
    enlaces.append(url)

enlaces[0]

'https://neoauto.com/auto/seminuevo/kia-sorento-2022-1844131'

## Extraccion de Informacion de segun el tipo de Vehicuklo y Condicion

Es necesario preguntar si quiere un auto, nuevo, usado o semenuevo.

In [115]:
while True:
    tipo = input("Es necesario preguntar si quiere un auto, autos (1), motos (2) o camiones (3)")

    if tipo == "1":
        tipo_vehiculo = "autos"
        break
    elif tipo == "2":
        tipo_vehiculo = "motos"
        break
    elif tipo == "3":
        tipo_vehiculo = "camiones"  
        break  
    
# while True:
#     tipo = input("Es necesario preguntar si quiere un auto que tipo de auto: camioneta (1), sedan (2), hatchback (3), pick-up (4), vans (5)")

#     if tipo == "1":
#         categoria = "camionetas-suv"
#         break
#     elif tipo == "2":
#         categoria = "sedan"
#         break
#     elif tipo == "3":
#         categoria = "hatchback" 
#         break  
#     elif tipo == "4":
#         categoria = "pick-up" 
#         break  
#     elif tipo == "5":
#         categoria = "vans" 
#         break  
#     elif tipo == "6":
#         categoria = "deportivo" 
#         break  


while True:
    tipo = input("Es necesario preguntar si quiere un auto, nuevo (1), usados (2) o seminuevos (3)")

    unidad = ""
    if tipo == "1":
        unidad = "nuevos"
        break
    elif tipo == "2":
        unidad = "usados"
        break
    elif tipo == "3":
        unidad = "seminuevos"
        break  

print(f"Se extraerán datos de: {tipo_vehiculo} - {unidad}")

Se extraerán datos de: autos - nuevos


Construccion de enlace

## Extraccion de Datos

In [116]:
# usados = range(1, 182)
# nuevo = range(1, 6)
# seminuevos = range(1, 44)

paginas = []

if unidad == "usados":
    for i in  range(1, 182):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "nuevos":
    for i in  range(1, 7):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })

if unidad == "seminuevos":
    for i in  range(1, 44):
        url = f"https://neoauto.com/venta-de-{tipo_vehiculo}-{unidad}?page={i}"
        paginas.append({
            "url": url,
            "tipo": unidad,
            "tipo_vehiculo": tipo_vehiculo
        })


# paginas[:10]
print(f"Se extraerán datos de {len(paginas)} páginas")

Se extraerán datos de 6 páginas


In [117]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()


neo_autos = []

for enlace in tqdm(paginas, desc="Páginas procesadas", unit="vehiculo"):
    
    tipo = enlace["tipo"]
    link_pagina = enlace["url"]
    tipo_vehiculo = enlace["tipo_vehiculo"]
    
    driver.get(link_pagina)
    
    BRANDS = sorted([
        "Alfa Romeo","Aston Martin","Land Rover","Mercedes Benz","Great Wall",
        "Volkswagen","Chevrolet","Toyota","Hyundai","Kia","Nissan","BMW","Audi",
        "Peugeot","Renault","Mazda","Subaru","Mitsubishi","Suzuki","Haval","JAC",
        "Changan","Geely","DFSK","Chery","Volvo","Porsche","Jaguar","Jeep","Ram",
        "Dodge","Ford","Seat","Skoda","Mini","BYD","MG","GMC","Lexus","Citroën","Citroen"
    ], key=len, reverse=True)


    ## NOMBRE, MARCA, MODELO, AÑO
    
    encabezado = driver.find_elements(By.CLASS_NAME, "c-results__header")

    titulo_auto = []
    marca_auto = []
    modelo_auto = []
    año_auto = []

    for enc in encabezado:
        titulo = enc.text
        titulo_ = titulo.split("\n", 1)[0].strip()
        for brand in BRANDS:
            if brand in titulo_:
                año = titulo_.split()[-1]
                modelo = titulo_.replace(brand, "").replace(año, "").strip()

                titulo_auto.append(titulo_)
                marca_auto.append(brand)    
                modelo_auto.append(modelo)
                año_auto.append(año)
                
    
    ## PRECIO
    
    precio = driver.find_elements(By.CLASS_NAME, "c-results-mount__price")
    precio_auto = []
    for prec in precio:
        costo = prec.text
        precio_auto.append(costo)

    precio_auto[0]
    
    ## TAGS
    
    tag =  driver.find_elements(By.CLASS_NAME, "c-results-tag__stick")
    tags = []
    for det in tag:
        descripcion = det.text
        tags.append(descripcion)
    
    ## DETALLE DE AUTO
        
    detalles = driver.find_elements(By.CLASS_NAME, "c-results-details__description")

    detalle_auto = []
    combustible_auto = []
    transmision_auto = []
    caja_auto = []
    kilometraje_auto = []
    ubicacion_auto = []

    for detalle in detalles:

        detalle_text = detalle.text

        # normaliza y separa líneas no vacías
        t = re.sub(r'[\r\xa0]', ' ', detalle_text).strip()
        lines = [l.strip() for l in t.splitlines() if l.strip()]

        # 1) combustible y transmisión (vienen en la 1ra línea separados por "|")
        combustible = transmision_raw = None
        if lines:
            p0 = re.split(r'\s*\|\s*', lines[0])  # acepta con o sin espacios: "Gasolina|Automática - Secuencial"
            combustible = p0[0].title() if p0 else None
            transmision_raw = p0[1] if len(p0) > 1 else None

        # 1.1) separa transmisión en tipo y caja si viene "Automática - Secuencial"
        tipo_transmision = caja = None
        if transmision_raw:
            parts = [p.strip() for p in transmision_raw.split("-")]
            tipo_transmision = parts[0]
            caja = parts[1] if len(parts) > 1 else None

        # 2) kilometraje (busca en todo el texto)
        km_m = re.search(r'(\d[\d.,]*)\s*kms?', t, flags=re.I)
        kilometraje_km = int(re.sub(r'[^\d]', '', km_m.group(1))) if km_m else None

        # 3) ubicación (primera línea con coma que no sea "Kms")
        ubicacion = next((l for l in lines if ',' in l and 'km' not in l.lower()), None)

        # 4) “modelo/etiqueta” (última línea)
        modelo_line = lines[-1] if lines else None

        detalle_auto.append(detalle_text)
        combustible_auto.append(combustible)
        transmision_auto.append(tipo_transmision)
        caja_auto.append(caja)
        kilometraje_auto.append(kilometraje_km)
        ubicacion_auto.append(ubicacion)
    
    ## URL DE AUTO
    
    url_auto = []
    enlace = driver.find_elements(By.CLASS_NAME, "c-results__link")   
    for link in enlace:
        url = link.get_attribute("href")    
        url_auto.append(url)
    
    ## ALMACENAMIENTO DE DATOS
    
    zip_list = zip(titulo_auto, marca_auto, modelo_auto, año_auto, precio_auto, tags, detalle_auto, combustible_auto, transmision_auto, caja_auto, kilometraje_auto, ubicacion_auto, url_auto)
    
    data_auto_page = []
    for titulo_auto_, marca_auto_, modelo_auto_, año_auto_, precio_auto_, tags_, detalle_auto_, combustible_auto_, transmision_auto_, caja_auto_, kilometraje_auto_, ubicacion_auto_, url_auto_ in zip_list:
        data_auto_page.append({
            "titulo": titulo_auto_,
            "tipo_vehiculo": tipo_vehiculo,
            "marca": marca_auto_,
            "modelo": modelo_auto_,
            "año": año_auto_,
            "precio": precio_auto_,
            "detalle": detalle_auto_,
            "combustible": combustible_auto_,
            "transmision": transmision_auto_,
            "caja": caja_auto_,
            "kilometraje": kilometraje_auto_,
            "ubicacion": ubicacion_auto_,
            "url_auto":url_auto_,
            "tags": tags_,
            "tipo": tipo,
            "pagina": link_pagina
        })
        
    neo_autos.extend(data_auto_page)
    

driver.quit()
    
# print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")
    

Páginas procesadas: 100%|██████████| 6/6 [00:15<00:00,  2.61s/vehiculo]


In [121]:
neo_autos[:2]

[{'titulo': 'Hyundai Accent 2024',
  'tipo_vehiculo': 'autos',
  'marca': 'Hyundai',
  'modelo': 'Accent',
  'año': '2024',
  'precio': 'US$ 17,990',
  'detalle': 'Gas GLP|Automática\nLima, Lima\nGARANTIA VIGENTE NEW ACCENT FULL GLP',
  'combustible': 'Gas Glp',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje': None,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/hyundai-accent-2024-1846381',
  'tags': 'PREMIUM',
  'tipo': 'nuevos',
  'pagina': 'https://neoauto.com/venta-de-autos-nuevos?page=1'},
 {'titulo': 'Hyundai Creta Grand 2025',
  'tipo_vehiculo': 'autos',
  'marca': 'Hyundai',
  'modelo': 'Creta Grand',
  'año': '2025',
  'precio': 'US$ 25,990',
  'detalle': 'Gas GLP|Automática\nLima, Lima\nGARANTÍA VIGENTE CRETA GRAND FULL GLP 2025',
  'combustible': 'Gas Glp',
  'transmision': 'Automática',
  'caja': None,
  'kilometraje': None,
  'ubicacion': 'Lima, Lima',
  'url_auto': 'https://neoauto.com/auto/nuevo/hyundai-creta-grand-2025-1846397

In [118]:
print(f"Se extrajeorn informacion de {len(neo_autos)} unidades")

Se extrajeorn informacion de 70 unidades


In [ ]:
neo_autos_df = pd.DataFrame(neo_autos)
neo_autos_df

,titulo,tipo_vehiculo,marca,modelo,año,precio,detalle,combustible,transmision,caja,kilometraje,ubicacion,url_auto,tags,tipo,pagina
0,Kia Sorento 2022,autos,Kia,Sorento,2022,"US$ 27,900",Gasolina|Automática - Secuencial\nKilometraje:...,Gasolina,Automática,Secuencial,33800.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/kia-sorento...,UNICO DUEÑO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
1,Land Rover Discovery 2018,autos,Land Rover,Discovery,2018,"US$ 41,000",Gasolina|Automática - Secuencial\nKilometraje:...,Gasolina,Automática,Secuencial,54000.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/land-rover-...,DE LUJO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
2,Jeep Compass 2023,autos,Jeep,Compass,2023,"US$ 28,500","Gasolina|Automática\nKilometraje:\n25,700 Kms\...",Gasolina,Automática,None,25700.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/jeep-compas...,SEMINUEVO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
3,Hyundai Creta Grand 2024,autos,Hyundai,Creta Grand,2024,"US$ 25,490",Gasolina|Automática\nKilometraje:\n18 Kms\nLim...,Gasolina,Automática,None,18.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/hyundai-cre...,COMO NUEVO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
4,Ram Rampage 2025,autos,Ram,page,2025,"US$ 30,500","Gasolina|Automática\nKilometraje:\n9,200 Kms\n...",Gasolina,Automática,None,9200.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/ram-rampage...,SEMINUEVO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,Nissan Kicks 2021,autos,Nissan,Kicks,2021,"US$ 53,900","Híbrido|Automática\nKilometraje:\n56,058 Kms\n...",Híbrido,Automática,None,56058.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/audi-q8-201...,DE LUJO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
553,Hyundai Santa Fe 2023,autos,Hyundai,Santa Fe,2023,"US$ 32,900","Gasolina|Automática\nKilometraje:\n46,938 Kms\...",Gasolina,Automática,None,46938.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/hyundai-san...,DE LUJO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
554,Audi A4 2017,autos,Audi,A4,2017,"US$ 22,500","Gasolina|Automática\nKilometraje:\n40,910 Kms\...",Gasolina,Automática,None,40910.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/audi-a4-201...,DE LUJO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...
555,Jeep Renegade 2018,autos,Jeep,Renegade,2018,"US$ 17,900","Gasolina|Automática\nKilometraje:\n86,559 Kms\...",Gasolina,Automática,None,86559.0,"Lima, Lima",https://neoauto.com/auto/seminuevo/bmw-318i-20...,COMO NUEVO,seminuevos,https://neoauto.com/venta-de-autos-seminuevos?...


In [ ]:
neo_autos_df